# softmax和分类模型
内容包含：  
1. softmax回归的概念
2. 获取Fashion-MNIST数据集
3. softmax回归模型实现，对图像数据进行分类

# softmax概念
- 分类问题  
一个简单的图像分类问题，输入图像高宽各2像素，色彩为灰度。  
图像的4像素分别记作$x_1,x_2,x_3,x_4$。  
假设真实标签有三类，分别为猫、狗、鸡，对应的离散值为$y_1=1,y_2=2,y_3=3$。  
- 权重矢量
$$o_1=x_1w_11+x_2w_21+x_3w_31+x_4w_41+b_1$$
$$o_2=x_1w_12+x_2w_22+x_3w_32+x_4w_42+b_2$$
$$o_3=x_1w_13+x_2w_23+x_3w_33+x_4w_43+b_3$$
- 神经网络图  
softmax回归同线性回归一样，也是一个单层神经网络。由于每个输出$o_1,o_2,o_3$的计算都要依赖于所有的输入$x_1,x_2,x_3,x_4$，softmax回归的输出层也是一个全连接层。  
![soft max neural network](https://staticcdn.boyuai.com/materials/2020/02/15/HOfTeZu5l4-dFL5XRA9y1.png!png)
既然分类问题需要得到离散的预测输出，一个简单的办法是将输出值$o_i$当作预测类别是$i$的置信度，并将值最大的输出所对应的类作为预测输出，即输出$\mathop{arg\,maxo_i}\limits_{i}$。  
例如，如果$o_1,o_2,o_3$分别为$0.1,10,0.1$，由于$o_2$最大，那么预测类别为2，其代表猫。  
- 输出问题  
直接使用输出层的输出有两个问题：  
    1. 一方面，由于输出层的输出值的范围不确定，难以直观上判断这些值的意义。
    2. 另一方面，由于真实标签是离散值，这些离散值与不确定范围的输出值之间的误差难以衡量。  
softmax运算符(softmax operator)解决了以上两个问题。它通过下式将输出值变换成值为正且和为1的概率分布：
$$\hat{y_1},\hat{y_2},\hat{y_3}=softmax(o_1,o_2,o_3)$$
其中
$$\hat{y_1}=\frac{exp(o_1)}{\sum_{i=1}^3exp(o_i)}$$
$$\hat{y_2}=\frac{exp(o_2)}{\sum_{i=1}^3exp(o_i)}$$
$$\hat{y_3}=\frac{exp(o_3)}{\sum_{i=1}^3exp(o_i)}$$
容易看出$\hat{y_1}+\hat{y_2}+\hat{y_3}=1$且$0\leq\hat{y_1},\hat{y_2},\hat{y_3}\leq1$，因此$\hat{y_1},\hat{y_2},\hat{y_3}$是一个合法的概率分布。此外，
$$\mathop{arg\,maxo_i}\limits_{i}=\mathop{arg\,max\hat{y_i}}\limits_{i}$$
因此softmax运算不改变预测类别输出。  
- 计算效率
    - 单样本矢量计算表达式
    $$W=
    \begin{bmatrix}
     w_11 & w_12 & w_13\\
     w_21 & w_22 & w_23\\
     w_31 & w_32 & w_33\\
     w_41 & w_42 & w_43\\
    \end{bmatrix},\,
     b=\begin{bmatrix} b_1 & b_2 & b_3 \end{bmatrix}
    $$
设高和宽分别为2个像素的图像样本$i$的特征为
$$x^{(i)}=\begin{bmatrix} x^{(i)}_1 & x^{(i)}_2 & x^{(i)}_3 & x^{(i)}_4 \end{bmatrix}$$
输出层的输出为
$$o^{(i)}=\begin{bmatrix} o^{(i)}_1 & o^{(i)}_2 & o^{(i)}_3 \end{bmatrix}$$
预测为猫、狗或鸡的概率分布为
$$\hat{y}^{(i)}=\begin{bmatrix} \hat{y}^{(i)}_1 & \hat{y}^{(i)}_2 & \hat{y}^{(i)}_3 \end{bmatrix}$$
softmax回归对样本$i$分类的矢量计算表达式为
$$o^{(i)}=x^{(i)}W+b$$
$$\hat{y}^{(i)}=softmax(o^{(i)})$$
- 小批量矢量计算表达式
为了进一步提升计算效率，通常对小批量数据做矢量计算。广义上讲，给定一个小批量样本，其批量大小为$n$，输入个数（特征数）为$d$，输出个数（类别墅）为$q$。设批量特征为$X\in \mathbb{R}^{n\times d}$。假设softmax回归的权重和偏差参数分别为$W\in \mathbb{R}^{d\times q}$和$b\in \mathbb{R}^{1\times q}$。softmax回归的矢量计算表达式为
$$O=XW+b$$
$$\hat{Y}=softmax(O)$$
其中的加法运算使用了广播机制，$O,\hat{Y}\in \mathbb{R}^{n\times q}$且这两个矩阵的第$i$行分别为样本$i$的输出$O^{(i)}$和概率分布$\hat{y}^{(i)}$。

# 交叉熵损失函数
对于样本$i$，我们构造向量$y_{(i)}\in \mathbb{R}^{q}$，使其第$y_{(i)}$（样本$i$类别的离散数值）个元素为1，其余为0。这样我们的训练目标可以设为使预测概率分布$\hat{y}^{(i)}$尽可能接近真实的标签概率分布$y^{(i)}$。
- 平方损失估计
$$Loss=|\hat{y}^{(i)}-y^{(i)}|^2/2$$
- 交叉熵（cross entropy）更适合衡量两个概率分布差异
$$H(y^{(i)},\hat{y}^{(i)})=-\sum_{j=1}^qy^{(i)}_jlog\hat{y}^{(i)}_{j}$$
其中$y^{(i)}_j$是向量$y^{(i)}$中非0即1的元素，需要注意将它与样本$i$类别的离散值，即$y^{(i)}$区分。在上式中，我们知道向量$y^{(i)}$中只有第$y^{(i)}$个元素$y^{(i)}_{y^{(i)}}$为1，其余全为0，于是$H(y^{(i)},\hat{y}^{(i)})=-log\hat{y}^{(i)}_{y^{(i)}}$。也就是说，交叉熵只关心对正确类别的预测概率，因为只要其值最勾搭，就可以确保分类结果正确。当然，遇到一个样本有多个标签时，例如图像里含有不止一个物体时，并不能做这一步简化。但即便对于这种情况，交叉熵同样只关心对图像中出现的物体类别的预测概率。  
假设训练数据集的样本数为$n$，交叉熵损失函数定义为
$${\scr{l}}(\Theta)=\frac{1}{n}\sum_{i=1}^nH(y^{(i)},\hat{y}^{(i)})$$

# 获取Fashion-MNIST数据集
使用torchvision：
1. .datasets：一些加载数据的函数和常用的数据集接口
2. .models：常用的模型结构（含预训练模型），例如AlexNet,VGG,ResNet;
3. .transforms：常用的图片变换
4. .utils： 其他方法等

In [8]:
# 导入库
%matplotlib inline
from IPython import display
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms
import time 

import sys
sys.path.append("/home/kesci/input")
import d2lzh1981 as d21

ModuleNotFoundError: No module named 'd2lzh1981'

###  获取数据集

In [10]:
mnist_train=torchvision.datasets.FashionMNIST(root='/home/kesci/input/FashionMNIST2065',train=True,download=True,transform=transforms.ToTensor())
mnist_test=torchvision.datasets.FashionMNIST(root='/home/kesci/input/FashionMNIST2065',train=False,download=True,transform=transforms.ToTensor())

KeyboardInterrupt: 

- root（string）– 数据集的根目录，其中存放processed/training.pt和processed/test.pt文件。
- train（bool, 可选）– 如果设置为True，从training.pt创建数据集，否则从test.pt创建。
- download（bool, 可选）– 如果设置为True，从互联网下载数据并放到root文件夹下。如果root目录下已经存在数据，不会再次下载。
- transform（可被调用 , 可选）– 一种函数或变换，输入PIL图片，返回变换之后的数据。如：transforms.RandomCrop。
- target_transform（可被调用 , 可选）– 一种函数或变换，输入目标，进行变换。

# softmax从零开始实现

In [1]:
import torch
import torchvision
import numpy as np
import sys
sys.path.append("/home/kesci/input")
import d2lzh as d2l

# 获取训练集和测试集

In [2]:
BATCH_SIZE=256
train_iter,test_iter=d2l.load_data_fashion_mnist(BATCH_SIZE,root='/home/kesci/input/FashionMNIST2065')

# 模型参数初始化

In [3]:
NUM_INPUTS= 784
NUM_OUTPUTS=10

W=torch.tensor(np.random.normal(0,0.01,(NUM_INPUTS,NUM_OUTPUTS)),dtype=torch.float)
b=torch.zeros(NUM_OUTPUTS,dtype=torch.float)

In [4]:
W.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

# 对多维Tensor按维度操作

In [24]:
X = torch.tensor([[1, 2, 3], [4, 5, 6]])
print("1)a",X.sum(dim=0, keepdim=True))  # dim为0，按照相同的列求和，并在结果中保留列特征
print("2)a",X.sum(dim=1, keepdim=True))  # dim为1，按照相同的行求和，并在结果中保留行特征
print("1)b",X.sum(dim=0, keepdim=False)) # dim为0，按照相同的列求和，不在结果中保留列特征
print("1)b",X.sum(dim=1, keepdim=False)) # dim为1，按照相同的行求和，不在结果中保留行特征

1)a tensor([[5, 7, 9]])
2)a tensor([[ 6],
        [15]])
1)b tensor([5, 7, 9])
1)b tensor([ 6, 15])


# 定义softmax操作
$$
\hat{y}_{j}=\frac{exp(o_{j})}{\sum_{i=1}^3exp(o_{i})}
$$

In [5]:
def softmax(X):
    X_exp = X.exp()
    partition = X_exp.sum(dim=1, keepdim=True)
    # print("X size is ", X_exp.size())
    # print("partition size is ", partition, partition.size())
    return X_exp / partition  # 这里应用了广播机制

In [86]:
X = torch.rand((2, 5))
X_prob = softmax(X)
print(X_prob, '\n', X_prob.sum(dim=1))

tensor([[0.2059, 0.1872, 0.3097, 0.1349, 0.1624],
        [0.1714, 0.1765, 0.1789, 0.2168, 0.2563]]) 
 tensor([1.0000, 1.0000])


# softmax回归模型
$$
{\bf{o}}^{(i)}={\bf{x}}^{(i)}{\bf{W}}+{\bf{b}},\\
{\bf{\hat{y}}}^{(i)}=softmax(o^{(i)}).
$$

In [6]:
def softmaxNet(X):
    return softmax(torch.mm(X.view((-1, num_inputs)), W) + b)

# 定义损失函数
$$
H({\bf{y}}^{(i)},{\bf{\hat{y}}}^{(i)})=-\sum_{j=1}^qy^{(i)}_{j}log\hat{y}^{(i)}_{j},\\
{\scr{l}}({\bf{\Theta}})=\frac{1}{n}\sum_{i=1}^nH({\bf{y}}^{(i)},{\bf{\hat{y}}}^{(i)}),\\
{\scr{l}}({\bf{\Theta}})=-\frac{1}{n}\sum_{i=1}^nlog\hat{y}^{(i)}_{y^{(i)}}
$$

torch.gather:parameters
- dim (python:int) – the axis along which to index
- index (LongTensor) – the indices of elements to gather

In [62]:
y_hat=torch.tensor([[0.1,0.3,0.6],[0.3,0.2,0.5]])
y=torch.LongTensor([0,2])
y.size()
# y.view(-1)
y_hat.gather(1,y.view(-1,1))


tensor([[0.1000],
        [0.5000]])

In [7]:
def cross_entropy(y_hat,y):
    return -torch.log(y_hat.gather(1,y.view(-1,1)))

# 定义准确率

In [8]:
def accuracy(y_hat,y):
    return (y_hat.argmax(dim=1)==y).float().mean().item()

In [79]:
print(accuracy(y_hat, y))

0.5


In [9]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
        n += y.shape[0]
    return acc_sum / n

In [10]:
print(evaluate_accuracy(test_iter,softmaxNet))

AttributeError: 'NDArray' object has no attribute 'view'

# 训练模型

In [75]:
num_epochs,lr=5,0.1
batch_size=256
def train_ch3(net,train_iter,test_iter,loss,num_epochs,batch_size,params=None,lr=None,optimizer=None):
    for epoch in range(num_epochs):
        train_l_sum,train_acc_sum,n=0.0,0.0,0
        for X,y in train_iter:
            y_hat=net(X)
            l=loss(y_hat,y).sum()
            
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
                    
            l.backward()
            if optimizer is None:
                d2l.sgd(params,lr,batch_size)
            else:
                optimizer.step()
            train_l_sum+=l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))
train_ch3(net, train_iter, test_iter, cross_entropy, num_epochs, batch_size, [W, b], lr)

AttributeError: 'NDArray' object has no attribute 'view'

#  模型预测

In [11]:
X, y = iter(test_iter).next()

true_labels = d2l.get_fashion_mnist_labels(y.numpy())
pred_labels = d2l.get_fashion_mnist_labels(net(X).argmax(dim=1).numpy())
titles = [true + '\n' + pred for true, pred in zip(true_labels, pred_labels)]

d2l.show_fashion_mnist(X[0:9], titles[0:9])

AttributeError: 'generator' object has no attribute 'next'

# softmax的PyTorch实现

In [13]:
# 加载各种包或者模块
import torch
from torch import nn
from torch.nn import init
import numpy as np
import sys
sys.path.append("/home/kesci/input")
import d2lzh as d2l

print(torch.__version__)

1.4.0


# 初始化参数和获取数据

In [14]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, root='/home/kesci/input/FashionMNIST2065')

# 定义网络模型

In [15]:
num_inputs = 784
num_outputs = 10

class LinearNet(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(num_inputs, num_outputs)
    def forward(self, x): # x 的形状: (batch, 1, 28, 28)
        y = self.linear(x.view(x.shape[0], -1))
        return y
    
# net = LinearNet(num_inputs, num_outputs)

class FlattenLayer(nn.Module):
    def __init__(self):
        super(FlattenLayer, self).__init__()
    def forward(self, x): # x 的形状: (batch, *, *, ...)
        return x.view(x.shape[0], -1)

from collections import OrderedDict
net = nn.Sequential(
        # FlattenLayer(),
        # LinearNet(num_inputs, num_outputs) 
        OrderedDict([
           ('flatten', FlattenLayer()),
           ('linear', nn.Linear(num_inputs, num_outputs))]) # 或者写成我们自己定义的 LinearNet(num_inputs, num_outputs) 也可以
        )

# 初始化模型参数

In [16]:
init.normal_(net.linear.weight, mean=0, std=0.01)
init.constant_(net.linear.bias, val=0)

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

# 定义损失函数

In [17]:
loss = nn.CrossEntropyLoss() # 下面是他的函数原型
# class torch.nn.CrossEntropyLoss(weight=None, size_average=None, ignore_index=-100, reduce=None, reduction='mean')

# 定义优化函数

In [19]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.1) # 下面是函数原型
# class torch.optim.SGD(params, lr=, momentum=0, dampening=0, weight_decay=0, nesterov=False)

# 训练

In [20]:
num_epochs = 5
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, None, None, optimizer)

AttributeError: 'NDArray' object has no attribute 'view'